In [3]:
import pandas as pd
import numpy as np

# Load the data
df = pd.read_csv(r'../resource/Mental-Health-Twitter-Preprocessed.csv')
df.head()

,post_id,post_created,post_text,user_id,followers,friends,favourites,statuses,retweets,URLs,Mentions,Hashtags,label
0,637894677824413696,Sun Aug 30 07:48:37 +0000 2015,it is just over years since i was diagnosed wi...,1013187241,84,211,251,837,0,False,False,"['#anxiety', '#depression']",1
1,637890384576778240,Sun Aug 30 07:31:33 +0000 2015,it is sunday i need a break so i am planning t...,1013187241,84,211,251,837,1,False,False,['#A14'],1
2,637749345908051968,Sat Aug 29 22:11:07 +0000 2015,awake but tired i need to sleep but my brain h...,1013187241,84,211,251,837,0,False,False,[],1
3,637696421077123073,Sat Aug 29 18:40:49 +0000 2015,bears make perfect gifts and are great for beg...,1013187241,84,211,251,837,2,True,True,"['#Retro', '#yay']",1
4,637696327485366272,Sat Aug 29 18:40:26 +0000 2015,it is hard to say whether packing lists are ma...,1013187241,84,211,251,837,1,False,False,"['#movinghouse', '#anxiety']",1


In [4]:
df['post_text'].head(15)

0     it is just over years since i was diagnosed wi...
1     it is sunday i need a break so i am planning t...
2     awake but tired i need to sleep but my brain h...
3     bears make perfect gifts and are great for beg...
4     it is hard to say whether packing lists are ma...
5                  making packing lists is my new hobby
6     at what point does keeping stuff for nostalgic...
7     currently in the findingboxesofrandomshit pack...
8     cannot be bothered to cook take away on the wa...
9     independent tv releases promo video for the fi...
10          also i have too much stuff way way too much
11      i never want to put one of these together again
12    moving stuff is bloomin knackering and there i...
13    back at the house moving stuff it is so peacef...
14     urgh anxiety ffs where does it come from breathe
Name: post_text, dtype: object

In [5]:
import nltk
# nltk.download('punkt')  # Download tokenizer models

from nltk.tokenize import word_tokenize

# Tokenize the post_text column
df['tokens'] = df['post_text'].apply(lambda x: word_tokenize(x.lower()))
df['tokens']

0        [it, is, just, over, years, since, i, was, dia...
1        [it, is, sunday, i, need, a, break, so, i, am,...
2        [awake, but, tired, i, need, to, sleep, but, m...
3        [bears, make, perfect, gifts, and, are, great,...
4        [it, is, hard, to, say, whether, packing, list...
                               ...                        
19761         [a, day, without, sunshine, is, like, night]
19762    [borens, laws, when, in, charge, ponder, when,...
19763    [the, flow, chart, is, a, most, thoroughly, ov...
19764    [ships, are, safe, in, harbor, but, they, were...
19765    [black, holes, are, where, god, is, dividing, ...
Name: tokens, Length: 19766, dtype: object

In [6]:
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def remove_stopword(tokens):
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return filtered_tokens

df['processed_tokens'] = df['tokens'].apply(remove_stopword)

# Convert POS tag from nltk format to wordnet format
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # default

def lemmatize_tokens(tokens):
    pos_tags = pos_tag(tokens)
    return [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags]


df['processed_tokens'] = df['processed_tokens'].apply(lemmatize_tokens)
df[['tokens','processed_tokens']]

,tokens,processed_tokens
0,"[it, is, just, over, years, since, i, was, dia...","[year, since, diagnose, today, take, moment, r..."
1,"[it, is, sunday, i, need, a, break, so, i, am,...","[sunday, need, break, plan, spend, little, tim..."
2,"[awake, but, tired, i, need, to, sleep, but, m...","[awake, tire, need, sleep, brain, idea]"
3,"[bears, make, perfect, gifts, and, are, great,...","[bear, make, perfect, gift, great, beginner, g..."
4,"[it, is, hard, to, say, whether, packing, list...","[hard, say, whether, pack, list, make, life, e..."
...,...,...
19761,"[a, day, without, sunshine, is, like, night]","[day, without, sunshine, like, night]"
19762,"[borens, laws, when, in, charge, ponder, when,...","[borens, law, charge, ponder, trouble, delegat..."
19763,"[the, flow, chart, is, a, most, thoroughly, ov...","[flow, chart, thoroughly, oversold, piece, pro..."
19764,"[ships, are, safe, in, harbor, but, they, were...","[ship, safe, harbor, never, mean, stay]"


In [7]:
# Save
label_col = df.pop('label')

df['label'] = label_col

df.to_csv('../resource/Mental-Health-Twitter-Tokenized.csv', index=False)